In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import os


In [2]:
TRAIN_DATA_PATH = "old_data/train/"
VAL_DATA_PATH = "old_data/val/"
TEST_DATA_PATH =  "old_data/test/"

In [3]:
# data transform, you can add different transform methods and resize image to any size
img_size = 200
data_transform = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

# data augmentation
train_transform = transforms.Compose([
                                      transforms.Resize((img_size,img_size)),
                                      transforms.RandomRotation(40),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(0, shear=0.2),
                                      transforms.RandomAffine(0, translate=(0.2, 0.2)),
                                      transforms.RandomAffine(0, scale=(1, 1.2)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

])

# dataset = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_transform)
train_dataset = datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=train_transform)
val_dataset = datasets.ImageFolder(root=VAL_DATA_PATH, transform=data_transform)

# spilt data into train and validation and the total number of image is 4276. You can decide the number of images
# you want to use to do training and validation.
TOTAL_SIZE = len(os.listdir(TRAIN_DATA_PATH + "/NORMAL")) + len(
    os.listdir(TRAIN_DATA_PATH + "/INFECTED")
)

# spilt your data into train and val
ratio = 0.7
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))

# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])


# you can use different batch size
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=500, shuffle=True,  num_workers=4)
# print(train_dataset)
# print(train_dataset.class_to_idx)

In [4]:
# I have written the function for you this time, but it's strongly recommended that you 
# understand how to do training and validation

def train(model, data_loader, optimizer, epoch, verbose=True):
    model.train()
    loss_avg = 0.0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        # loss function, you can use other loss function if you want
        loss   = F.cross_entropy(output, target)
        loss_avg = loss.item()
        
        # do back propagation
        loss.backward()
        optimizer.step()
        verbose_step = len(data_loader) // 10
        if batch_idx % verbose_step == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
    return loss_avg / len(data_loader)

def valid(model, data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            valid_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item() 

        valid_loss /= len(data_loader.dataset)
        print('\nValid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            valid_loss, correct, len(data_loader.dataset),
            100. * correct / len(data_loader.dataset)))
    return float(correct) / len(data_loader.dataset)

In [5]:
############## Build the model here ##########
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
    
        super(ConvNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, 3),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.drop = nn.Dropout(0.5)
        
        self.fc = nn.Sequential(
            nn.Linear(10*10*128, 512),
            nn.ReLU(),
            nn.Linear(512, 2))
        
        self.sigmoid = nn.Sigmoid()
        
        

    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1) # flatten
        out = self.drop(out)
        out = self.fc(out)
        out = self.sigmoid(out) # can discard?
        return out




In [6]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


if not torch.cuda.is_available():
    print("NO GPU!!!!!")

In [ ]:
####################  implement your optimizer ###################################
## yo can use any training methods if you want (ex:lr decay, weight decay.....)

model = ConvNet() # instantiate model

if torch.cuda.is_available():
    model.cuda()

lr = 1e-4
# optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer = optim.Adam(model.parameters(), lr=lr)
#optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

# start training
epochs = 100
acc = 0.0
for epoch in range(epochs):
    model.train()
    train(model, train_data_loader, optimizer, epoch)
    accuracy = valid(model, val_data_loader)
    if accuracy > acc:
        acc = accuracy
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "model.pth")

Train Epoch: 0 [0/3701 (0%)]	Loss: 0.693088
Train Epoch: 0 [256/3701 (7%)]	Loss: 0.648213
Train Epoch: 0 [512/3701 (14%)]	Loss: 0.645158
Train Epoch: 0 [768/3701 (21%)]	Loss: 0.620534
Train Epoch: 0 [1024/3701 (28%)]	Loss: 0.584444
Train Epoch: 0 [1280/3701 (34%)]	Loss: 0.566207
Train Epoch: 0 [1536/3701 (41%)]	Loss: 0.564071
Train Epoch: 0 [1792/3701 (48%)]	Loss: 0.508755
Train Epoch: 0 [2048/3701 (55%)]	Loss: 0.539855
Train Epoch: 0 [2304/3701 (62%)]	Loss: 0.586701
Train Epoch: 0 [2560/3701 (69%)]	Loss: 0.594514
Train Epoch: 0 [2816/3701 (76%)]	Loss: 0.524200
Train Epoch: 0 [3072/3701 (83%)]	Loss: 0.563262
Train Epoch: 0 [3328/3701 (90%)]	Loss: 0.555449


In [ ]:
test_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                    ])

test_data = datasets.ImageFolder(root=TEST_DATA_PATH,transform=test_transform)
test_data_loader  = data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4) 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model so that you don't need to train the model again
test_model = torch.load("model.pth").to(device)

In [ ]:
def test(model,data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        bs = test_data_loader.batch_size
        result = []
        for i, (data, target) in enumerate(test_data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            arr = pred.data.cpu().numpy()
            for j in range(pred.size()[0]):
                file_name = test_data.samples[i*bs+j][0].split('/')[-1]
                result.append((file_name,pred[j].cpu().numpy()[0]))
    return result

In [ ]:
result = test(test_model,test_data_loader)

# Write results to csv

In [ ]:
with open ('ID_result.csv','w') as f:
    f.write('ID,label\n')
    for data in result:
        f.write(data[0]+','+str(data[1])+'\n')